In [ ]:
#!jupyter nbconvert --to=python DSi_WOA_Comparison_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class DSicomp:
    '''
    class DSicomp
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 WOAvar='i_an',
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 depth_limit = 100):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit

        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        #from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from load_interp_WOA_python3 import WOAdata
        from plot_Taylor_normalized import plt_Taylor_norm

        # load FESOM data -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        NCfesom = self.resultpath + '/DSi.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM DSi {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'DSi [mmol Si m$^{-3}$]' 

        # load data -------------------------------------------------------------------------------------
        print('Processing '+resultpath)
        
        DSifesom = pf.get_data(resultpath, "DSi", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)


        # load WOA data  -------------------------------------------------------------------------------------
        WOA_input = WOAdata(self.runname,self.resultpath,self.meshpath,self.ncpath,self.WOAvar, get_overview=False)
        dsi_int = WOA_input.woa_int    
        #print(np.shape(din_int))
        
        labelwoa = 'World Ocean Atlas DSi'
        unitwoa = 'DSi [$\mu$mol Si m$^{-3}$]' #i_an:units = "micromoles_per_liter"
        
        print('***\nUnit WOA: '+unitwoa+'\nUnit FESOM: '+unitfesom+'\n***')

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        dsi_int_ma = np.copy(dsi_int)
        dsi_int_ma[DSifesom == 0] = 0

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(True):
                    print('\nInput depth = {0}, plotting at level depth = {1} m\nFESOM min = {2:4.6f}, max = {3:4.6f}\nWOA min = {4:4.6f}, max = {5:4.6f}'.format(
                        d,plot_depth,
                        np.nanmin(DSifesom[:,i]),np.nanmax(DSifesom[:,i]),
                        np.nanmin(dsi_int_ma[:,i]),np.nanmax(dsi_int_ma[:,i])))
                    print('Min of difference WOA-FESOM: {0}'.format(np.nanmin(dsi_int_ma[:,i]-DSifesom[:,i])))

                pf.plot(mesh, [dsi_int_ma[:,i],DSifesom[:,i], dsi_int_ma[:,i]-DSifesom[:,i]], 
                        rowscol= (1,3),
                        levels = (-40,120,81),
                        units=unitwoa + '\n at depth = {0} m'.format(plot_depth), 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        cmap = self.cmap,
                        titles=[labelwoa, labelfesom, 'WOA - FESOM'],figsize = (20,20)
                       )

                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig('{0}{1}_DSi_WOA_at{2}m_{3}to{4}.png'.format(
                        self.savepath, self.runname, plot_depth, str(years[0]), str(years[-1])), 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False) 
            
            # statistics  -------------------------------------------------------------------------------------
            # preparation of datasets
            if np.isnan(np.min(dsi_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DSifesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)
                
                title = 'Taylor Diagram for DSi at {0} m'.format(plot_depth)
                plt_Taylor_norm(dsi_int[i,:],DSifesom[i,:],mask=True,title=title,plot=False)


                if False:#(self.savefig==True):
                    plt.savefig('{0}{1}_DSi_WOA_Taylor_at{2}m_{3}to{4}.png'.format(
                        self.savepath, self.runname, plot_depth, str(years[0]), str(years[-1])), 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False) 
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            # get level depth index closest to depth_limit
            i_depth_limit = pf.ind_for_depth(self.depth_limit, mesh)
            
            # corresponding level depth
            depth_limit_level = - mesh.zlev[i_depth_limit]
            
            # corresponding layer depth (where concentrations are defined):
            depth_limit_layer = - WOA_input.layer_depths[i_depth_limit]
            
            # print overview of chosen depth
            print('DSi as mean over depth \nwith max layer depth = {0} \n(level depth = {1}m, mesh index {2})'.format(
                depth_limit_layer,depth_limit_level,i_depth_limit))
            
            # mean over depth
            DSifesom_mean = np.mean(DSifesom[:,:i_depth_limit], axis = 1)
            dsi_int_ma_mean = np.mean(dsi_int_ma[:,:i_depth_limit], axis = 1)

            print('\nFESOM min = {0:4.4f}, max = {1:4.4f}\nWOA min = {2:4.4f}, max = {3:4.4f}'.format(
                    np.nanmin(DSifesom_mean),np.nanmax(DSifesom_mean),
                    np.nanmin(dsi_int_ma_mean),np.nanmax(dsi_int_ma_mean)))
            print('Min of difference WOA-FESOM: {0:4.4f}'.format(np.nanmin(dsi_int_ma_mean-DSifesom_mean)))
    
            fig_data = pf.plot(mesh, [dsi_int_ma_mean,DSifesom_mean, dsi_int_ma_mean-DSifesom_mean], 
                    rowscol= (1,3),
                    levels = (-40,120,81),
                    units=unitwoa+'\nMean over depth', 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    cmap = self.cmap,
                    titles=['World Ocean Atlas: Interpolated DSi for initialization', 
                            'Mean over depth (max={2}m)\nFESOM Dissolved Silicate ({0}-{1})'.format(
                                first_year,last_year,depth_limit_layer),
                            'Initialization fields minus FESOM'],
                    figsize = (20,20)
                   )
        
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'DSi_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
                
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(dsi_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DSifesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

            title = 'Taylor Diagram for DSi \n(mean over depth, max = {0}m)'.format(depth_limit_layer)
            plt_Taylor_norm(dsi_int_ma_mean,DSifesom_mean,mask=True,title=title)
                
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'DSi_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
        


In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runname      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo20'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'


    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1965
    last_year  = 1966

    # WOA ------------------------------------------------------------------------------------    
    ncpath = '/work/ollie/projects/MarESys/evaluation/woa13_all_i00_01.nc'
    #!ncdump -h $ncpath
    
    # now test mean over depth:
    test = DSicomp(runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                   mapproj = 'pc',
                   layerwise=False,
                   depth_limit = 200,
                   savefig=False)


In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # now test layerwise:
    test = DSicomp(runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                   mapproj = 'pc',
                   layerwise=True,
                   depth_array = (0,1000),
                   savefig=False)